## Entendimiento de los datos

Vamos a trabajar con el conjunto de datos Churn-Modeling descargado de ***Kaggle***. Este conjunto de datos contiene detalles de los clientes de un banco y la variable objetivo es una variable binaria que refleja el hecho de si el cliente dejó el banco (cerró su cuenta) o si continúa siendo un cliente.

Las características capturan información sociodemográfica del cliente, información de productos financieros, el comportamiento y balance de su cuenta. Las características son las siguientes:

* **CustomerId**: Id único para identificar el cliente.
* **Surname**: Apellido del cliente.
* **CreditScore**: Puntaje de crédito del cliente.
* **Geography**: País al que pertenece el cliente.
* **Gender**: Genero.
* **Age**: Edad.
* **Tenure**: Número de años que el cliente ha estado en el banco.
* **Balance**: Saldo bancario del cliente.
* **NumOfProducts**: Número de productos bancarios que utiliza el cliente.
* **HasCrCard**: Si el cliente tiene tarjeta de crédito con el banco.
* **IsActiveMember**: Si el cliente es miembro activo del banco o no.
* **EstimatedSalary**: Salario estimado en dólares.
* **Exited**: 1-Si el cliente cerró la cuenta con el banco; 0-Si el cliente es retenido.


### Carga de módulos

In [13]:
# !pip install missingno
# Python library that provides the ability to understand the distribution of missing values through visualizations
# (Heat maps / Bar charts)

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [43]:
churn = pd.read_csv("https://raw.githubusercontent.com/stivenlopezg/DS-ONLINE-76/master/data/churn-modeling.csv",dtype={"CustomerId": "category"})
churn.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,0


In [16]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CustomerId       10000 non-null  category
 1   Surname          10000 non-null  object  
 2   CreditScore      10000 non-null  int64   
 3   Geography        9980 non-null   object  
 4   Gender           10000 non-null  object  
 5   Age              10000 non-null  int64   
 6   Tenure           10000 non-null  int64   
 7   Balance          10000 non-null  float64 
 8   NumOfProducts    10000 non-null  int64   
 9   HasCrCard        10000 non-null  object  
 10  IsActiveMember   10000 non-null  object  
 11  EstimatedSalary  9988 non-null   float64 
 12  Exited           10000 non-null  int64   
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 1.3+ MB


In [44]:
# Estadística descriptiva

churn.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9988.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,100066.908601,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,57519.993379,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,50910.677500,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,100185.240000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,199992.480000,1.000000


In [45]:
churn.describe(exclude="number")

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember
count,10000,10000,9980,10000,10000,10000
unique,10000,2932,3,2,2,2
top,15815690,Smith,France,Male,Yes,Yes
freq,1,32,5008,5457,7055,5151


#Datos missing
Vamos a mirar si hay datos missing en nuestro set de datos

In [46]:
# Conteo
churn.isna().sum()

CustomerId          0
Surname             0
CreditScore         0
Geography          20
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary    12
Exited              0
dtype: int64

In [47]:
# Lo mismo, en porcentaje
churn.isna().mean() * 100

CustomerId         0.00
Surname            0.00
CreditScore        0.00
Geography          0.20
Gender             0.00
Age                0.00
Tenure             0.00
Balance            0.00
NumOfProducts      0.00
HasCrCard          0.00
IsActiveMember     0.00
EstimatedSalary    0.12
Exited             0.00
dtype: float64

### Preprocesamiento

* Variables numéricas:
    * Atípicos
    * Imputar
    * Escalar
    * Discretizar (Opcional)

* Variables categóricas:
    * Imputar
    * Codificar (OHE, LabelEncoder, u OrdinalEncoder)

In [48]:
#Decidimos que estas columnas confunden al modelo o no tienen correlacion con la var que queremos predecir
cols_to_drop = ["CustomerId", "Surname"]

# axis = 0 se refiere a rows, axis=1 a columns
# Para major claridad, se puede usar axis='index' para rows y axis='columns'

churn.drop(labels=cols_to_drop, axis='columns', inplace=True)

churn.sample(n=1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6342,581,France,Female,37,10,104255.03,1,Yes,No,86609.37,0


In [50]:
#Lista de features de tipo numérico
numerical_features = churn.select_dtypes(include="number").columns.tolist()

# Removemos la variable target, que queremos predecir
numerical_features.remove("Exited")
numerical_features

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [92]:
#Lista de features catégoricos, a partir de excluír los que no son number
categorical_features = churn.select_dtypes(exclude="number").columns.tolist()
categorical_features

['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

In [52]:
#Pop: method to delete any element from the dataframe
exited = churn.pop("Exited")

# train_data, test_data, train_label, test_label

#test_size refers to the data percentage used for testing. If 0.3, 0.7 is the one used for training.
X_train, X_test, y_train, y_test = train_test_split(churn, exited,
                                                    test_size=0.3)

In [53]:
X_train.isna().sum()

CreditScore         0
Geography          15
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary    10
dtype: int64

#### Preprocesamiento sobre los features numéricos

In [26]:
#Imputacion - Rellenar los datos faltantes - estrategias:
 # mean: replace missing values using the mean along each column
 # median: replace missing values using the median along each column
 # most_frequent: replace missing using the most frequent value along each column.
    # Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.
 # constant: replace missing values with fill_value.

# selecciona de X_train solo aquellas columnas que están contenidas en la lista "numerical_features list
imputer_num = SimpleImputer(strategy="median").fit(X_train[numerical_features])

#Valor de la mediana con la que imputó cada feature.
imputer_num.statistics_

array([6.5200000e+02, 3.7000000e+01, 5.0000000e+00, 9.6889925e+04,
       1.0000000e+00, 9.9953470e+04])

In [54]:
# Transform: aplica sobre las columnas numericas los valores resultado de la
# imputación anterior - en este caso la media - sobre los valores faltantes
X_train.loc[:, numerical_features] = imputer_num.transform(X_train[numerical_features])

#Vemos que EstimatedSalary ya no tiene nulos. Resta trabajar sobre Geography, que es un valor categórico
X_train.isna().sum()

/home/gery/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


CreditScore         0
Geography          15
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary     0
dtype: int64

In [114]:
# Escalar: util cuando sigue una distribución normal.
# StandardScaler estandariza un feature al restarle el promedio y luego dividir
# por la desviacion estandar, obteniendo "unidades" de desviacion estandar.
# Así, el promedio de la distribucion es 0 y alrededor del 68% de los valores
# estarán entre -1 and 1.

scaler = StandardScaler().fit(X_train[numerical_features])

X_train.loc[:, numerical_features] = scaler.transform(X_train[numerical_features])

In [57]:
# Podemos ver que se aplicó a todas las columnas numericas y que ya los valores
# ya no tienen la misma interpretacion que al momento de importar el dataset
X_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
4816,-0.516465,France,Male,-0.951182,1.030985,-1.232479,0.816121,No,Yes,-1.139811
3634,0.319106,France,Male,-1.524462,0.686700,1.301340,-0.905654,No,No,0.825750
3681,0.267527,France,Female,0.099833,-1.034723,1.141403,-0.905654,Yes,No,-0.088723
3543,0.257211,France,Male,0.959753,1.030985,0.937908,-0.905654,No,Yes,1.378158
4229,-0.598991,France,Female,-0.473448,1.375270,-1.232479,0.816121,Yes,Yes,-1.388300


#### Preprocesamiento Variables categóricas

In [83]:
# Imputacion

# Sobre datos categóricos, la estrategia puede ser la "más frecuente" o el uso de una "constante"

imputer_cat = SimpleImputer(strategy="most_frequent").fit(X_train[categorical_features])

# Para cada variable categórica, arma un array con el valor más frecuente
imputer_cat.statistics_

X_train.loc[:, categorical_features] = imputer_cat.transform(X_train[categorical_features])

#Geography pasó a tener cero valores faltantes. Segun vimos antes, el top value era Francia
X_train.isna().sum()

/home/gery/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [105]:
# One Hot Encoder
#Para variables nominales sin jerarquía, crea una columna por cada uno de los valores encontrado
#asignando una variable dummy - que puede tomar dos posibles valores: 1 o 0 - para indicar
#la presencia o ausencia de esa categoria en esa observación

## no le está gustando la lista categorical_features como param de columns,
#me tira error: raise TypeError("Input must be a list-like for parameter `columns`")
#X_train = pd.get_dummies(data=X_train, columns=categorical_features)

#Se lo saqué. La doc dice que por default, toda columna de dtype object o categorical se lo aplica
#si el parámetro no se especifica.
X_train = pd.get_dummies(data=X_train)
X_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_No,HasCrCard_Yes,IsActiveMember_No,IsActiveMember_Yes
4816,-0.516465,-0.951182,1.030985,-1.232479,0.816121,-1.139811,1,0,0,0,1,1,0,0,1
3634,0.319106,-1.524462,0.686700,1.301340,-0.905654,0.825750,1,0,0,0,1,1,0,1,0
3681,0.267527,0.099833,-1.034723,1.141403,-0.905654,-0.088723,1,0,0,1,0,0,1,1,0
3543,0.257211,0.959753,1.030985,0.937908,-0.905654,1.378158,1,0,0,0,1,1,0,0,1
4229,-0.598991,-0.473448,1.375270,-1.232479,0.816121,-1.388300,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2060,0.896784,-0.091261,0.342416,1.116979,0.816121,1.708526,0,0,1,0,1,0,1,1,0
8787,0.401631,3.252875,-1.379008,0.511031,-0.905654,1.177239,0,0,1,0,1,1,0,0,1
6018,0.834890,-0.855635,0.686700,-1.232479,0.816121,0.744462,1,0,0,0,1,1,0,0,1
6498,-0.382361,-1.142275,-0.001869,0.777849,-0.905654,1.718758,1,0,0,1,0,0,1,1,0


### Set de Test
#### Preprocesamiento Variables Numéricas

In [108]:
#Conteo de nulos por columna
X_test.isna().sum()


CreditScore        0
Geography          5
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    2
dtype: int64

Imputación

In [109]:
# imputer_num: Definida durante el training, array con la imputación a aplicar sobre numericos
X_test.loc[:, numerical_features] = imputer_num.transform(X_test[numerical_features])

#Columnas originalmente con valores faltantes, ya no tienen más
X_test.isna().sum()

/home/gery/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


CreditScore        0
Geography          5
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

Escalado sobre el set de Test

In [124]:
#Scaler

X_test.loc[:, numerical_features] = scaler.transform(X_test[numerical_features])

#No veo que aplique el scaler a los features numericos.
X_test.head()

/home/gery/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
2529,716.0,France,Female,29.0,10.0,8.794639e+04,1.0,Yes,Yes,182531.74
7352,726.0,France,Female,38.0,4.0,1.218073e-17,2.0,No,No,6787.48
7053,755.0,Germany,Male,43.0,6.0,1.650485e+05,3.0,Yes,No,16929.41
7490,654.0,France,Female,35.0,2.0,9.086580e+04,1.0,Yes,Yes,86764.46
9117,703.0,France,Female,32.0,6.0,1.218073e-17,2.0,No,No,33606.52


### Set de Test
#### Preprocesamiento Variables categóricas

In [123]:
X_test.loc[:, categorical_features] = imputer_cat.transform(X_test[categorical_features])

#Geography pasó a tener cero valores faltantes. Segun vimos antes, el top value era Francia
X_test.isna().sum()

/home/gery/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

#### One Hot Encoder

In [129]:
X_test = pd.get_dummies(data=X_test)
X_test.sample(4)


,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_No,HasCrCard_Yes,IsActiveMember_No,IsActiveMember_Yes
1906,786.0,29.0,4.0,1.218073e-17,2.0,103372.79,1,0,0,1,0,0,1,1,0
9272,739.0,42.0,2.0,1.416429e+05,2.0,172149.76,1,0,0,0,1,0,1,1,0
1916,543.0,48.0,1.0,1.009005e+05,1.0,33310.72,0,1,0,0,1,1,0,1,0
4409,756.0,41.0,6.0,1.490499e+05,1.0,50422.36,0,1,0,0,1,1,0,0,1
